In [ ]:
import os
import pandas as pd
from azure.storage.filedatalake import DataLakeServiceClient
from io import BytesIO

# Ladda .env OM den inte redan är laddad
if 'STORAGE_ACCOUNT_NAME' not in os.environ:
    env_file = '.env'
    if os.path.exists(env_file):
        with open(env_file, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#') and '=' in line:
                    key, value = line.split('=', 1)
                    value = value.strip().strip('"').strip("'")
                    os.environ[key.strip()] = value
        print(f"✅ Laddade .env från: {env_file}")

import config as cfg

def read_parquet_from_adls(path):
    svc = DataLakeServiceClient(
        account_url=f"https://{cfg.STORAGE_ACCOUNT_NAME}.dfs.core.windows.net",
        credential=cfg.STORAGE_ACCOUNT_KEY
    )
    fs = svc.get_file_system_client(cfg.CONTAINER_NAME)
    fc = fs.get_file_client(path)
    data = fc.download_file().readall()
    buf = BytesIO(data)
    return pd.read_parquet(buf)

df = read_parquet_from_adls("curated/planned/df_planned_latest.parquet")
df.head()